# 14 - Using External ASE Objects (Calculators/Optimizers)

### In this tutorial we will learn how to use external ASE calculators and optimizers within the architector framework.

**(A)** Using arbitrary methods to perform energy/relaxations.

**(B)** How to use example ASE calculator (MACE-MP-0) within Architector complex construction.

**(C)** How to use arbirtrary ASE optimizers within Architector.

To get started - make sure to install the MACE-MP-0 package and model:

See: https://github.com/ACEsuit/mace

In [1]:
from architector import view_structures, convert_io_molecule
from architector.io_calc import CalcExecutor
from executorlib import Executor
import numpy as np

Now, Let's generate a structure (UFF) for an arbitary SMILES string and distort the coordinates by a random distance:

In [2]:
arb_smiles = 'c1ccccc1CON'
mol = convert_io_molecule(arb_smiles)
mol.ase_atoms.set_positions(mol.ase_atoms.get_positions() + np.random.random(mol.ase_atoms.get_positions().shape) / 2)
view_structures(mol)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## How to use arbirtrary ASE optimizers within Architector.

Here, we can directly use other ASE optimizers as well, the default is LBFGSLineSearch. So Let's USE BFGS.

In [3]:
def get_parameters():
    from ase.optimize import BFGS
    from mace.calculators import mace_mp # Import Mace ASE calculator
    calc = mace_mp(
        model="medium", 
        dispersion=True,
        default_dtype="float32"
    )
    return {"parameters": {
        'ase_opt_method':BFGS, # Directly pass the optimizer object
        'calculator':calc,
        'assemble_method':'UFF', # Use UFF for assembly
        'full_method':'custom',
        'debug':True # Use debug to show that it is using BFGS instead of LBFGSLineSearch
    }}

In [4]:
def build(input_dict, parameters):
    from architector import build_complex
    input_dict["parameters"] = parameters
    return build_complex(input_dict)

In [5]:
inputDict = {
    'core': {'metal':'Fe','coreCN': 6}, # Specify metal coordination number (CN)
    'ligands':['water'] * 6, # Specify what is filling the coordination environment
} 
with Executor(max_workers=1, threads_per_core=2, backend="flux", block_allocation=True, init_function=get_parameters) as exe:
    f = exe.submit(build, input_dict=inputDict)
    fe_h2o_out = f.result()

In [6]:
view_structures(fe_h2o_out)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Summary: In this tutorial we learned how to use external ASE calculators and optimizers within the architector framework. Including:

**(A)** Using arbitrary methods to perform energy/relaxations.

**(B)** How to use example ASE calculator (MACE-MP-0) within Architector complex construction.

**(C)** How to use arbirtrary ASE optimizers within Architector.

In [7]:
!flux resource list

     STATE NNODES   NCORES    NGPUS NODELIST
      free      1        4        0 cmpc06
 allocated      0        0        0 
      down      0        0        0 


In [8]:
!flux jobs -a 

       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO
   ƒ2cyT4dMH janssen  python     CD      1      1    42.6s cmpc06
   ƒ21AZJjwM janssen  python     CD      1      1   40.64s cmpc06
    ƒYA51bew janssen  python     CD      1      1   31.63s cmpc06
